## Lab: Quantum Teleportation

Quantum teleportation is a protocol that enables the transmission of an arbitrary quantum state from one party (Alice) to another (Bob) using only classical communication and a shared entangled pair. Alice performs a Bell-basis measurement on her qubit and her half of the entangled pair, producing two classical bits. She sends these bits to Bob over a classical channel. Depending on Alice’s measurement results, Bob applies one of four possible corrections to his half of the entangled pair, which recovers the original quantum state. Importantly, the quantum state is transferred without the physical transmission of the qubit itself.

### Task

Alice prepares a random quantum state and teleports it to Bob using the teleportation protocol. The task is to create a circuit that demonstrates this process.

### Expected Output

The program should draw the circuit for each teleportation instance, display Alice’s two classical measurement bits, show Bob’s test result, and report whether the teleportation was successful or failed.

### Optional Challenge

Extend the program so that multiple random states are teleported in one run, each with its own verification step.  


In [ ]:
from IPython.display import display

from qiskit import QuantumCircuit, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit.visualization import circuit_drawer
import math, random

def alice_encode(theta, phi):
    qc = QuantumCircuit(3,2)
    
    # alice prepares |S⟩ on qubit 0
    qc.ry(theta, 0)
    qc.rz(phi, 0)

    # Alice create bell pair on qubits 1-2
    qc.h(1)
    qc.cx(1, 2)
    qc.barrier()
    
    # Alice perfoms the  bell-basis measurement on (0,1)
    qc.cx(0, 1)
    qc.h(0)
    qc.barrier()
    qc.measure(0,0) # alice[0] = msg bit
    qc.measure(1,1) # alice[1] = bell bit 
    qc.barrier()
    return qc

def bob_decode(qc):
    """
    corrections based on Alice's measurement
      00 -> I (none)
      01 -> Z
      10 -> X
      11 -> XZ
    """
    with qc.if_test((0, 1)):
        qc.z(2)
    with qc.if_test((1, 1)):
        qc.x(2)
    qc.barrier()
    return qc

def bob_test(qc,theta, phi):
    # uncompute and measure bob (should be 0)
    qc.rz(-phi, 2)
    qc.ry(-theta, 2)
    
    test = ClassicalRegister(1, "test")
    qc.add_register(test) 
    qc.measure(2,test)
    return qc
 
# ------------------------------------------------
#                main program
# ------------------------------------------------

#-- prepare test data as rotation angles --
theta = random.random() * math.pi
phi = random.random() * 2 * math.pi

qc = alice_encode(theta, phi)

# ------- transmission lines -------

qc = bob_decode(qc)
qc = bob_test(qc,theta, phi)

# --  setup simulator and run ---
simulator = AerSimulator()
result = simulator.run(qc, shots=1).result()
counts = result.get_counts(qc)

# --  display results ---
print(f"Teleportation state: theta={theta:.3f}, phi={phi:.3f}")
"""
Note: Qiskit reverses the order in get_counts():
- Bob's test bit was added last, so it appears first
- Alice's register prints as q1q0 
"""
key = next(iter(counts))
test, alice_bits  = key.split() 
if test == '0':  
    print(f"Alice bits: {alice_bits} | Bob's test bit {test}: Transmission OK.")
else:
    print(f"Alice bits: {alice_bits} | Bob's test bit {test}: Transmission FAILED.")

display(circuit_drawer(qc, output="mpl"))
